In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import time

# All normal velocity components are 0 - since all are "walls", no inlets
velocity_top_wall = 1  # Top wall velocity
velocity_bottom_wall = 0  # Bottom wall velocity
velocity_left_wall = 0  # Left wall velocity
velocity_right_wall = 0  # Right wall velocity

total_time = 1  # Total simulation time
kinematic_viscosity = 0.01  # Fluid's kinematic viscosity
density = 1  # Fluid density

grid_points_x = 24  # Number of grid points in x-direction
grid_points_y = 24  # Number of grid points in y-direction
domain_length_x = 1  # Length of the domain in x-direction
domain_length_y = 1  # Length of the domain in y-direction
step_size_x = domain_length_x / grid_points_x
step_size_y = domain_length_y / grid_points_y

time_step = min(0.25 * (min(step_size_x, step_size_y) ** 2) / kinematic_viscosity, (4 * kinematic_viscosity) / (velocity_top_wall ** 2)) / 2
total_steps = math.ceil(total_time / time_step)
time_step = total_time / total_steps

# Initialize velocities and pressure
u_current = np.zeros([grid_points_y + 2, grid_points_x + 2])
v_current = np.zeros([grid_points_y + 2, grid_points_x + 2])
pressure_current = np.zeros([grid_points_y + 2, grid_points_x + 2])
pressure_rhs = np.zeros([grid_points_y + 2, grid_points_x + 2])

u_intermediate = np.zeros([grid_points_y + 2, grid_points_x + 2])
v_intermediate = np.zeros([grid_points_y + 2, grid_points_x + 2])

u_solution, v_solution, pressure_solution = [], [], []
u_solution.append(u_current)
v_solution.append(v_current)
pressure_solution.append(pressure_current)

# Pressure-Poisson equation coefficients
coeff_pressure = np.zeros([grid_points_y + 2, grid_points_x + 2])
coeff_east = np.ones([grid_points_y + 2, grid_points_x + 2]) * (1 / (step_size_x ** 2))
coeff_west = np.ones([grid_points_y + 2, grid_points_x + 2]) * (1 / (step_size_x ** 2))
coeff_north = np.ones([grid_points_y + 2, grid_points_x + 2]) * (1 / (step_size_y ** 2))
coeff_south = np.ones([grid_points_y + 2, grid_points_x + 2]) * (1 / (step_size_y ** 2))

# Set boundary coefficients
coeff_west[:, 1] = 0  # Left boundary
coeff_east[:, -2] = 0  # Right boundary
coeff_north[-2, :] = 0  # Top boundary
coeff_south[1, :] = 0  # Bottom boundary

coeff_pressure = -(coeff_west + coeff_east + coeff_north + coeff_south)

# Pressure-Poisson equation solver
def solve_pressure_poisson(p_guess, rhs):
    iteration = 0
    error = 1e3
    tolerance = 1e-6
    max_iterations = 100
    relaxation_factor = 1.1
    
    while error > tolerance and iteration < max_iterations:
        p_prev = p_guess.copy()
        for x in range(1, grid_points_x + 1):
            for y in range(1, grid_points_y + 1):
                rhs_term = rhs[y, x] - (coeff_east[y, x] * p_guess[y, x + 1] +
                                        coeff_west[y, x] * p_guess[y, x - 1] +
                                        coeff_north[y, x] * p_guess[y + 1, x] +
                                        coeff_south[y, x] * p_guess[y - 1, x])
                p_guess[y, x] = relaxation_factor * (rhs_term / coeff_pressure[y, x]) + (1 - relaxation_factor) * p_guess[y, x]
        error = np.linalg.norm(p_guess - p_prev, 2)
        iteration += 1
    return p_guess, error

start_time = time.time()
simulation_time = 0
while simulation_time < total_time:
    u_previous = u_solution[-1]
    v_previous = v_solution[-1]
    pressure_previous = pressure_solution[-1]
    
    # Boundary conditions for u
    u_previous[:, 1] = 0  # Left wall
    u_previous[:, -1] = 0  # Right wall
    u_previous[-1, :] = 2 * velocity_top_wall - u_previous[-2, :]  # Top wall ghost cell
    u_previous[0, :] = 2 * velocity_bottom_wall - u_previous[1, :]  # Bottom wall ghost cell
    
    # Boundary conditions for v
    v_previous[-1, :] = 0  # Top wall
    v_previous[1, :] = 0  # Bottom wall
    v_previous[:, 0] = 2 * velocity_left_wall - v_previous[:, 1]  # Left wall ghost cell
    v_previous[:, -1] = 2 * velocity_right_wall - v_previous[:, -2]  # Right wall ghost cell
    
    u_next = u_previous.copy()
    v_next = v_previous.copy()
    
    # Predictor step
    for x in range(2, grid_points_x + 1):
        for y in range(1, grid_points_y + 1):
            u_east = 0.5 * (u_previous[y, x + 1] + u_previous[y, x])
            u_west = 0.5 * (u_previous[y, x] + u_previous[y, x - 1])
            u_north = 0.5 * (u_previous[y, x] + u_previous[y + 1, x])
            u_south = 0.5 * (u_previous[y, x] + u_previous[y - 1, x])
            v_north = 0.5 * (v_previous[y + 1, x] + v_previous[y + 1, x - 1])
            v_south = 0.5 * (v_previous[y, x] + v_previous[y, x - 1])
            convection = -(u_east * u_east - u_west * u_west) / step_size_x - (u_north * v_north - u_south * v_south) / step_size_y
            diffusion = kinematic_viscosity * ((u_previous[y, x + 1] - 2 * u_previous[y, x] + u_previous[y, x - 1]) / (step_size_x ** 2) + 
                                               (u_previous[y + 1, x] - 2 * u_previous[y, x] + u_previous[y - 1, x]) / (step_size_y ** 2))
            u_intermediate[y, x] = convection * time_step + diffusion * time_step + u_previous[y, x]
    
    for x in range(1, grid_points_x + 1):
        for y in range(2, grid_points_y + 1):
            v_east = 0.5 * (v_previous[y, x] + v_previous[y, x + 1])
            v_west = 0.5 * (v_previous[y, x] + v_previous[y, x - 1])
            v_north = 0.5 * (v_previous[y, x] + v_previous[y + 1, x])
            v_south = 0.5 * (v_previous[y, x] + v_previous[y - 1, x])
            u_west = 0.5 * (u_previous[y, x] + u_previous[y - 1, x])
            u_east = 0.5 * (u_previous[y, x + 1] + u_previous[y - 1, x + 1])
            convection = -(u_east * v_east - u_west * v_west) / step_size_x - (v_north * v_north - v_south * v_south) / step_size_y
            diffusion = kinematic_viscosity * ((v_previous[y, x + 1] - 2 * v_previous[y, x] + v_previous[y, x - 1]) / (step_size_x ** 2) + 
                                               (v_previous[y + 1, x] - 2 * v_previous[y, x] + v_previous[y - 1, x]) / (step_size_y ** 2))
            v_intermediate[y, x] = convection * time_step + diffusion * time_step + v_previous[y, x]
    
    # Solve PPE
    for x in range(1, grid_points_x + 1):
        for y in range(1, grid_points_y + 1):
            pressure_rhs[y, x] = (u_intermediate[y, x + 1] - u_intermediate[y, x]) / step_size_x + (v_intermediate[y + 1, x] - v_intermediate[y, x]) / step_size_y
    pressure_rhs /= time_step
    pressure_next, solver_error = solve_pressure_poisson(pressure_previous, pressure_rhs)
    
    # Corrector step
    u_next[1:-1, 2:-1] = u_intermediate[1:-1, 2:-1] - (time_step / (density * step_size_x)) * (pressure_next[1:-1, 2:-1] - pressure_next[1:-1, 1:-2])
    v_next[2:-1, 1:-1] = v_intermediate[2:-1, 1:-1] - (time_step / (density * step_size_y)) * (pressure_next[2:-1, 1:-1] - pressure_next[1:-2, 1:-1])
    
    u_solution.append(u_next)
    v_solution.append(v_next)
    pressure_solution.append(pressure_next)
    
    simulation_time += time_step
    progress = simulation_time / total_time
    update_progress_bar(int(simulation_time / time_step), total_steps)

update_progress_bar(total_steps, total_steps)
elapsed_time = time.time() - start_time
print(f"\nSimulation completed in {round(elapsed_time, 3)} seconds")


In [2]:
u_solution

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,